# Virtual Piano Tutor

## Project Description

This is a notebook for tracking my progress on VPT...

- Best Classifier as of 11/30
    - SVM {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}

## TODO List

- TODAY
    - Decide on RDF model to keep for rest of project
    - Work on RDF data and annotations
    - Add results to file
    - Rewrite RDF for GridSearchCV
        - Extend RDF
    - Work on ideas for paper
        - Visualizations
    - Play with CAE
    - How to automate this...
    - Windowing/Summarizing
    
- DONE
    - ~~Organize RDF data~~
    - ~~Generate data from already extracted hands...~~
    - ~~Get notebook running on Compute Canada~~
    - ~~Get data on Compute Canada~~
    - ~~Setup CAE to deal with hand images~~
    - ~~setup data for training autoencoder on LH and RH~~
    - ~~Train Autoencoder for LH and Rh~~
    

- Bad Segmentation
    - p3c - left hand (not terrible)
    - p1s - right hand (shouldn't use)
    - p5a - Both could use some work but still caputures most of the left hand (RH not so good...)
    - p5c - not good (left hand passable...)
    
- Add noise to CAE
    - http://scikit-image.org/docs/dev/api/skimage.util.html#random-noise
    
- ~~Multiple Participants~~
    - ~~have one holdout set participant~~
        - ~~Test with p1&2 training p3 testing, then p1&3...~~
    - ~~have one holdout set exercise~~

- Test with RH too

- Windowing data
    - Summarize data for classification
    - Majority Voting (or with probabilities)

- Look for other features
    - Others??
    - ~~Autoencoder features~~
    - ~~HONV~~
    
- Work on hand segmentation
   - See p1e for bad examples
   - How to validate segmentation?
       - Statistical analysis on length and width ratios
       
- Visualize !!!
    - Input 
    - Results !!!
        - F Scores
        - Accuracy
        - Try weighted instead of macro




- Finish Project Description

- ~~Turn into functions~~
    
- ~~Verify Segmentation~~
    - have only done basic verification
    
- ~~FIRST THING: Test by ignoring training data (p1s) and then using train_test_split on recordings~~
    - ~~Data should be ready for spliting~~
    
- ~~Remove data from testing to find culprit~~
    
- ~~Track my progress better !!! (duh through notebooks!)~~

# Setup

## Libraries

In [1]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2

from vpt.features.features import *
import vpt.utils.image_processing as ip
import vpt.settings as s
import vpt.hand_detection.depth_context_features as dcf

%load_ext autoreload
%autoreload 2

## Some helper functions

### Load/Save Data Set

In [2]:
def load_data(testing_p, M, radius, feature_type, data_type):
    base = "data/posture/extracted/"
    data_path = os.path.join(base, "{}_M{}_rad{:0.2f}_{}_".format(testing_p, M, radius, feature_type))
    data = np.load(data_path + data_type + "_data_combined.npz")    
    return data

## Project Setup

### Generate or Load Data

In [3]:
#### Load data for a single paricipant
M = 5
radius = .15
feature_type = "hog"

data = load_data("all_participants", M, radius, feature_type, "train")
print("X LH", data["X_lh"].shape, "y LH", data["y_lh"].shape, data["vis_lhs"].shape)
print("X RH", data["X_rh"].shape, "y RH", data["y_rh"].shape, data["vis_rhs"].shape)
print("Filenames", data["filenames"].shape)

X LH (15818, 1089) y LH (15818,) (15818, 180, 180)
X RH (15818, 1089) y RH (15818,) (15818, 180, 180)
Filenames (15818,)


In [52]:
## using p6 for validation
r = re.compile('p1')
vmatch = np.vectorize(lambda x:bool(r.search(x)))
val_p = vmatch(data['filenames'])

X_lh = data['X_lh'][~val_p]
y_lh = data['y_lh'][~val_p]
X_rh = data['X_rh'][~val_p]
y_rh = data['y_rh'][~val_p]
filenames = data['filenames'][~val_p]


X_lh_test = data['X_lh'][val_p]
y_lh_test = data['y_lh'][val_p]
X_rh_test = data['X_rh'][val_p]
y_rh_test = data['y_rh'][val_p]
filenames_test = data['filenames'][val_p]

print("Cross Val Data:")
print(X_lh.shape, y_lh.shape)
print(X_rh.shape, y_rh.shape)
print(filenames.shape)
print()
print("Validation Data")
print(X_lh_test.shape, y_lh_test.shape)
print(X_rh_test.shape, y_rh_test.shape)
print(filenames_test.shape)

Cross Val Data:
(12262, 1089) (12262,)
(12262, 1089) (12262,)
(12262,)

Validation Data
(3556, 1089) (3556,)
(3556, 1089) (3556,)
(3556,)


In [53]:
groups = np.zeros_like(filenames, dtype=int)

for p in ["p1", "p3", "p4", "p6"]:
    p_num = int(p[1])
    groups[np.where(np.char.find(filenames, p) != -1)] = p_num
    
print(groups.shape)
print(np.unique(groups))
print(groups)

(12262,)
[1 3 4]
[1 1 1 ..., 4 4 4]


# Classification

### Libraries

In [10]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, LeaveOneGroupOut
from imblearn.pipeline import Pipeline

from sklearn.decomposition import PCA

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN

## Load Data for Classification

In [54]:
## find all "static" data so we can ignore for now
r = re.compile('p[\d]s')

# remove p#s data
vmatch = np.vectorize(lambda x:bool(r.search(x)))
rem_static = vmatch(filenames)
rem_static_test = vmatch(filenames_test)

X_lh_train, y_lh_train, filenames_train, groups_train = X_lh[~rem_static], y_lh[~rem_static], filenames[~rem_static], groups[~rem_static]
X_rh_train, y_rh_train, filenames_train, groups_train = X_rh[~rem_static], y_rh[~rem_static], filenames[~rem_static], groups[~rem_static]

In [55]:
X_lh_val, y_lh_val, filenames_val = X_lh_test[~rem_static_test], y_lh_test[~rem_static_test], filenames_test[~rem_static_test]
X_rh_val, y_rh_val, filenames_val = X_rh_test[~rem_static_test], y_rh_test[~rem_static_test], filenames_test[~rem_static_test]

In [56]:
X_comb_train = np.vstack((X_lh_train, X_rh_train))
y_comb_train = np.hstack((y_lh_train, y_rh_train))
groups_comb_train = np.hstack((groups_train, groups_train))
filenames_comb_train = np.hstack((filenames_train, filenames_train))

X_comb_val = np.vstack((X_lh_val, X_rh_val))
y_comb_val = np.hstack((y_lh_val, y_rh_val))
filenames_comb_val = np.hstack((filenames_val, filenames_val))

In [57]:
print("Cross Validation Data")
print("LH:", X_lh_train.shape, y_lh_train.shape)
print(np.unique(y_lh_train, return_counts=True))
print("RH:", X_rh_train.shape, y_rh_train.shape)
print(filenames_train.shape)
print(groups_train.shape)
print(np.unique(y_rh_train, return_counts=True))
print("Combined:", X_comb_train.shape, y_comb_train.shape)
print(filenames_comb_train.shape)
print(groups_comb_train.shape)
print(np.unique(y_comb_train, return_counts=True))
print()
print()
print("Validation Data")
print("LH:", X_lh_val.shape, y_lh_val.shape)
print(np.unique(y_lh_val, return_counts=True))
print("RH:", X_rh_val.shape, y_rh_val.shape)
print(np.unique(y_rh_val, return_counts=True))
print("Combined:", X_comb_val.shape, y_comb_val.shape)
print(np.unique(y_comb_val, return_counts=True))
print(filenames_comb_val.shape)

Cross Validation Data
LH: (9067, 1089) (9067,)
(array([0, 1, 2]), array([5088, 2911, 1068]))
RH: (9067, 1089) (9067,)
(9067,)
(9067,)
(array([0, 1]), array([7104, 1963]))
Combined: (18134, 1089) (18134,)
(18134,)
(18134,)
(array([0, 1, 2]), array([12192,  4874,  1068]))


Validation Data
LH: (3212, 1089) (3212,)
(array([0, 1]), array([3069,  143]))
RH: (3212, 1089) (3212,)
(array([0, 1, 2]), array([3042,  143,   27]))
Combined: (6424, 1089) (6424,)
(array([0, 1, 2]), array([6111,  286,   27]))
(6424,)


## Model Testing

### SVM

In [17]:
## Parameters for SVMs
steps = [('SMOTE', SMOTE()), ("SVC", SVC())]
param_grid = [
#   {'SVC__C': [1, 10, 100], 'SVC__kernel': ['linear'], 'SMOTE__kind': ['regular', 'borderline1', 'borderline2', 'svm']},
#   {'SVC__C': [1, 10, 100], 'SVC__gamma': [.0001, .001, .01, .1], 'SVC__kernel': ['rbf'], 'SMOTE__kind': ['regular', 'borderline1', 'borderline2', 'svm']},
  {'SVC__C': [10, 100], 'SVC__gamma': [.000005, .00001, .00005,], 'SVC__kernel': ['rbf'], 'SMOTE__kind': ['regular', 'borderline1', 'borderline2']},
 ]

# steps = [('SMOTE', SMOTEENN()), ("SVC", SVC())]
# param_grid = [
#   {'SVC__C': [1, 10, 100, 1000], 'SVC__kernel': ['linear'], 'SMOTE__smote': [SMOTE(kind='regular'), SMOTE(kind='borderline1'), SMOTE(kind='borderline2'), SMOTE(kind='svm')]},
#   {'SVC__C': [1, 10, 100, 1000], 'SVC__gamma': [.0001, .001, .01, .1], 'SVC__kernel': ['rbf'], 'SMOTE__smote': [SMOTE(kind='regular'), SMOTE(kind='borderline1'), SMOTE(kind='borderline2'), SMOTE(kind='svm')]},
#  ]


pipeline = Pipeline(steps)

scores = ['f1']
logo = LeaveOneGroupOut()

In [18]:
# Hyper Parameter Tuning
for score in scores:
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        print("## Tuning hyper-parameters for {}".format(score))
        print()

        if score is "accuracy":
            scoring = score
        else:
            scoring = '{}_macro'.format(score)      
        
        #### TRAIN COMBINED LH & RH
        clf_comb = GridSearchCV(pipeline, param_grid, cv=logo.split(X_comb_train, y_comb_train,groups=groups_comb_train), scoring=scoring, n_jobs=2, verbose=10)
        clf_comb.fit(X_comb_train, y_comb_train)

        print("Best Combined Parameters set found on data set:")
        print()
        print(clf_comb.best_params_)
        print()
        print("Grid scores on data set:")
        print()
        means = clf_comb.cv_results_['mean_test_score']
        stds  = clf_comb.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf_comb.cv_results_['params']):
            print("%0.3f (+/-%0.3f) for %r" % (mean, std, params))
        print()

## Tuning hyper-parameters for f1

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] SMOTE__kind=regular, SVC__C=10, SVC__gamma=5e-06, SVC__kernel=rbf 
[CV] SMOTE__kind=regular, SVC__C=10, SVC__gamma=5e-06, SVC__kernel=rbf 
[CV]  SMOTE__kind=regular, SVC__C=10, SVC__gamma=5e-06, SVC__kernel=rbf, total=14.1min
[CV] SMOTE__kind=regular, SVC__C=10, SVC__gamma=5e-06, SVC__kernel=rbf 
[CV]  SMOTE__kind=regular, SVC__C=10, SVC__gamma=5e-06, SVC__kernel=rbf, total=26.7min
[CV] SMOTE__kind=regular, SVC__C=10, SVC__gamma=1e-05, SVC__kernel=rbf 
[CV]  SMOTE__kind=regular, SVC__C=10, SVC__gamma=5e-06, SVC__kernel=rbf, total=12.2min
[CV] SMOTE__kind=regular, SVC__C=10, SVC__gamma=1e-05, SVC__kernel=rbf 
[CV]  SMOTE__kind=regular, SVC__C=10, SVC__gamma=1e-05, SVC__kernel=rbf, total=11.8min
[CV] SMOTE__kind=regular, SVC__C=10, SVC__gamma=1e-05, SVC__kernel=rbf 
[CV]  SMOTE__kind=regular, SVC__C=10, SVC__gamma=1e-05, SVC__kernel=rbf, total=10.6min
[CV] SMOTE__kind=regular, SVC__C=10, 

[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed: 296.6min


[CV]  SMOTE__kind=borderline2, SVC__C=10, SVC__gamma=5e-06, SVC__kernel=rbf, total=15.0min
[CV] SMOTE__kind=borderline2, SVC__C=10, SVC__gamma=1e-05, SVC__kernel=rbf 
[CV]  SMOTE__kind=borderline2, SVC__C=10, SVC__gamma=5e-06, SVC__kernel=rbf, total=30.7min
[CV] SMOTE__kind=borderline2, SVC__C=10, SVC__gamma=1e-05, SVC__kernel=rbf 
[CV]  SMOTE__kind=borderline2, SVC__C=10, SVC__gamma=1e-05, SVC__kernel=rbf, total=13.5min
[CV] SMOTE__kind=borderline2, SVC__C=10, SVC__gamma=1e-05, SVC__kernel=rbf 
[CV]  SMOTE__kind=borderline2, SVC__C=10, SVC__gamma=1e-05, SVC__kernel=rbf, total=13.4min
[CV] SMOTE__kind=borderline2, SVC__C=10, SVC__gamma=5e-05, SVC__kernel=rbf 
[CV]  SMOTE__kind=borderline2, SVC__C=10, SVC__gamma=1e-05, SVC__kernel=rbf, total=26.7min
[CV] SMOTE__kind=borderline2, SVC__C=10, SVC__gamma=5e-05, SVC__kernel=rbf 
[CV]  SMOTE__kind=borderline2, SVC__C=10, SVC__gamma=5e-05, SVC__kernel=rbf, total= 8.8min
[CV] SMOTE__kind=borderline2, SVC__C=10, SVC__gamma=5e-05, SVC__kernel=rbf

[Parallel(n_jobs=2)]: Done  54 out of  54 | elapsed: 433.1min finished


Best Combined Parameters set found on data set:

{'SMOTE__kind': 'regular', 'SVC__C': 100, 'SVC__gamma': 5e-05, 'SVC__kernel': 'rbf'}

Grid scores on data set:

0.169 (+/-0.160) for {'SMOTE__kind': 'regular', 'SVC__C': 10, 'SVC__gamma': 5e-06, 'SVC__kernel': 'rbf'}
0.198 (+/-0.158) for {'SMOTE__kind': 'regular', 'SVC__C': 10, 'SVC__gamma': 1e-05, 'SVC__kernel': 'rbf'}
0.246 (+/-0.131) for {'SMOTE__kind': 'regular', 'SVC__C': 10, 'SVC__gamma': 5e-05, 'SVC__kernel': 'rbf'}
0.248 (+/-0.133) for {'SMOTE__kind': 'regular', 'SVC__C': 100, 'SVC__gamma': 5e-06, 'SVC__kernel': 'rbf'}
0.268 (+/-0.104) for {'SMOTE__kind': 'regular', 'SVC__C': 100, 'SVC__gamma': 1e-05, 'SVC__kernel': 'rbf'}
0.291 (+/-0.061) for {'SMOTE__kind': 'regular', 'SVC__C': 100, 'SVC__gamma': 5e-05, 'SVC__kernel': 'rbf'}
0.145 (+/-0.182) for {'SMOTE__kind': 'borderline1', 'SVC__C': 10, 'SVC__gamma': 5e-06, 'SVC__kernel': 'rbf'}
0.178 (+/-0.163) for {'SMOTE__kind': 'borderline1', 'SVC__C': 10, 'SVC__gamma': 1e-05, 'SVC__kern

In [19]:
from sklearn.metrics import accuracy_score
y_comb_true, y_comb_pred = y_comb_val, clf_comb.predict(X_comb_val)

print("Comb Validatation Score:", accuracy_score(y_comb_true, y_comb_pred))
print("Comb Confustion Matrix:\n", confusion_matrix(y_comb_true, y_comb_pred))
print(classification_report(y_comb_true, y_comb_pred))
print()


Comb Validatation Score: 0.816981132075
Comb Confustion Matrix:
 [[3450  464    3]
 [ 456  880    0]
 [  15   32    0]]
             precision    recall  f1-score   support

          0       0.88      0.88      0.88      3917
          1       0.64      0.66      0.65      1336
          2       0.00      0.00      0.00        47

avg / total       0.81      0.82      0.81      5300




### Single Classifier

In [58]:
# steps = [('PCA', PCA(n_components=1500)), ('SMOTE', SMOTE(kind="borderline2")), ("SVC", SVC(C=100, gamma=.0001, kernel='rbf', probability=False))]
steps = [('SMOTE', SMOTE(kind="regular")), ("SVC", SVC(C=100, gamma=.00005, kernel='rbf', probability=False))]
pipeline = Pipeline(steps)
pipeline.fit(X_comb_train, y_comb_train)

Pipeline(memory=None,
     steps=[('SMOTE', SMOTE(k=None, k_neighbors=5, kind='regular', m=None, m_neighbors=10, n_jobs=1,
   out_step=0.5, random_state=None, ratio='auto', svm_estimator=None)), ('SVC', SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=5e-05, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [59]:
y_comb_true, y_comb_pred = y_comb_val, pipeline.predict(X_comb_val)
print("Comb Validatation Score:", accuracy_score(y_comb_true, y_comb_pred))
print("Comb Confustion Matrix:\n", confusion_matrix(y_comb_true, y_comb_pred))
print(classification_report(y_comb_true, y_comb_pred))

Comb Validatation Score: 0.416407222914
Comb Confustion Matrix:
 [[2424 3684    3]
 [  21  251   14]
 [  10   17    0]]
             precision    recall  f1-score   support

          0       0.99      0.40      0.57      6111
          1       0.06      0.88      0.12       286
          2       0.00      0.00      0.00        27

avg / total       0.94      0.42      0.54      6424



In [60]:
window_size = 10
y_comb_true_maj = []
for i in range(0, y_comb_true.size, window_size):
    u, counts = np.unique(y_comb_true[i:i+window_size], return_counts=True)
    pred = u[np.argmax(counts)]
    y_comb_true_maj.append(pred)
    
y_comb_true_maj = np.array(y_comb_true_maj)

In [61]:
y_comb_pred_maj = []
for i in range(0, y_comb_pred.size, window_size):
    u, counts = np.unique(y_comb_pred[i:i+window_size], return_counts=True)
    pred = u[np.argmax(counts)]
    y_comb_pred_maj.append(pred)
    
y_comb_pred_maj = np.array(y_comb_pred_maj)

In [62]:
print("rH Validatation Score:", accuracy_score(y_comb_true_maj, y_comb_pred_maj))
print("rH Confustion Matrix:\n", confusion_matrix(y_comb_true_maj, y_comb_pred_maj))
print(classification_report(y_comb_true_maj, y_comb_pred_maj))

rH Validatation Score: 0.429237947123
rH Confustion Matrix:
 [[250 362   0]
 [  1  26   1]
 [  1   2   0]]
             precision    recall  f1-score   support

          0       0.99      0.41      0.58       612
          1       0.07      0.93      0.12        28
          2       0.00      0.00      0.00         3

avg / total       0.95      0.43      0.56       643



### Cross Validation

In [36]:
from sklearn.model_selection import cross_val_score, cross_validate

In [37]:
X_comb_all_cv = np.vstack((X_comb_train, X_comb_val))
y_comb_all_cv = np.hstack((y_comb_train, y_comb_val))
filenames_comb_all_cv = np.hstack((filenames_comb_train, filenames_comb_val))

print("Combined:", X_comb_all_cv.shape, y_comb_all_cv.shape)
print(np.unique(y_comb_all_cv, return_counts=True))
print(filenames_comb_all_cv.shape)

Combined: (24558, 1089) (24558,)
(array([0, 1, 2]), array([18303,  5160,  1095]))
(24558,)


In [ ]:
groups_cv = np.zeros_like(filenames_comb_all_cv, dtype=int)

for p in ["p1", "p3", "p4", "p6"]:
    p_num = int(p[1])
    groups_cv[np.where(np.char.find(filenames_comb_all_cv, p) != -1)] = p_num
    
print(groups_cv.shape)
print(np.unique(groups_cv))
print(groups_cv)

In [ ]:
steps = [('SMOTE', SMOTE(kind="borderline2")), ("SVC", SVC(C=1, gamma=.001, kernel='rbf', probability=False))]
pipeline = Pipeline(steps)
scoring = ("f1_macro", "accuracy")
logo = LeaveOneGroupOut()
scores = cross_validate(pipeline, X_comb_all_cv, y_comb_all_cv, cv=logo.get_n_splits(groups=groups_cv), scoring=scoring, n_jobs=2, verbose=2)

In [ ]:
print(scores)

In [ ]:
for k, v in scores.items():
    print(k, v)

In [ ]:
steps = [('SMOTE', SMOTE(kind="borderline2")), ("SVC", SVC(C=1, gamma=.001, kernel='rbf', probability=False))]
pipeline = Pipeline(steps)
scoring = ("f1_macro", "accuracy")
logo = LeaveOneGroupOut()
scores = cross_validate(pipeline, X_comb_all_cv, y_comb_all_cv, cv=logo.split(X_comb_all_cv, y_comb_all_cv, groups=groups_cv), scoring=scoring, n_jobs=2, verbose=3)

In [ ]:
for k, v in scores.items():
    print(k, v)

In [ ]:
logo.split(X_comb_all_cv, y_comb_all_cv, groups=groups_cv)